In [1]:
import numpy as np
import hail as hl
from hail import methods
import scipy as sp
import pandas as pd
from math import sqrt, pi

## Create genetic data (and clean/process/edit, these cells are retired now, the code to run is in the last cell at the bottom of the notebook)

In [4]:
# Create genetic data and write to disk
bnm_mt = hl.balding_nichols_model(3, 100, 1000)
bnm_mt.write("balding_nichols_3_100_1000.mt")

2020-07-27 10:17:45 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 100 samples, and 1000 variants...
2020-07-27 10:17:45 Hail: INFO: Coerced sorted dataset
2020-07-27 10:17:48 Hail: INFO: wrote matrix table with 1000 rows and 100 columns in 8 partitions to balding_nichols_3_100_1000.mt


In [5]:
# Read first MatrixTable and clean

# entries are now calls: An object that represents an individual’s call at a genomic locus
mt = hl.read_matrix_table("balding_nichols_3_100_1000.mt")

# don't understand meaning of this: returns the count of non-reference alleles from each call
mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 

# Turn MatrixTable into Table

ht = mt.localize_entries("ent", "sample")

2020-07-27 10:19:25 Hail: WARN: Name collision: field 'sample' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['sample']


In [101]:
# retired - no longer using for data generation
def makeData(model_input, group_size):
    mt = hl.balding_nichols_model(*model_input)
    mt.write("balding_nichols_test.mt")
    mt = hl.read_matrix_table("balding_nichols_test.mt")
    mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 
    table = mt.localize_entries("ent", "sample")
    table = matrix_table_to_table_of_ndarrays(mt.n_alt, group_size, tmp_path='/tmp/test_table.ht')
#     table = table.key_by(hl.int32(table.row_group_number))
    return table
    
# data = makeData((3, 100, 1000), 4)

# (n, m) = (100, 1000)
# k = 50
# l = k + 2
# q = 0

# G = hl.nd.array(np.random.normal(0, 1, (n,l)))

## Grouping and NDArray methods from Tim and Dan

In [327]:
# Functions for operating with Tables of ndarrays in Hail (from Tim)

from hail.expr import Expression, ExpressionException, \
    expr_float64, expr_call, expr_any, expr_numeric, expr_array, \
    expr_locus, \
    analyze, check_entry_indexed, check_row_indexed, \
    matrix_table_source, table_source

# Only groups by rows, NOT COLUMNS
def matrix_table_to_table_of_ndarrays(field, group_size, tmp_path = '/tmp/nd_table.ht'):
    """

    The returned table has two fields: 'row_group_number' and 'ndarray'.

    Examples
    --------
    >>> ht = matrix_table_to_table_of_ndarrays(mt.GT.n_alt_alleles(), 100)

    Parameters
    ----------
    field
    group_size
    tmp_path

    Returns
    -------

    """
    mt = matrix_table_source('matrix_table_to_table_of_ndarrays/x', field)
    mt = mt.select_entries(x = field)
    ht = mt.localize_entries(entries_array_field_name='entries')
    # now ht.entries is an array of structs with one field, x

    # we'll also want to mean-impute/variance-normalize/etc here
    ht = ht.select(xs = ht.entries.map(lambda e: e['x']))
    # now ht.xs is an array of float64

    # now need to produce groups of G
    ht = ht.add_index()
    ht = ht.group_by(row_group_number= hl.int32(ht.idx // group_size)) \
        .aggregate(ndarray=hl.nd.array(hl.agg.collect(ht.xs)))
    # may require a .T on ndarray

    return ht.checkpoint(tmp_path, overwrite=True)

def chunk_ndarray(a, group_size):
    """Chunks a NDarray along the first axis in chunks of `group_size`.
    Parameters
    ----------
    a
    group_size
    -------

    """
    n_groups = a.shape[0] // group_size
    groups = []
    for i in range(a.shape[0] // group_size):
        start = i * group_size
        end = (i + 1) * group_size
        groups.append(a[start:end, :])
    return groups


# Concatenate the ndarrays with a blocked Table
def concatBlocked(A):
    blocks = A.ndarray.collect()
    big_mat = np.concatenate(blocks, axis=0)
    ht = ndarray_to_table([big_mat])
    
    block_shape = blocks[0].shape
    
    tup = ht.ndarray.collect()[0].shape
    assert (tup == (len(blocks) * block_shape[0], block_shape[1]))
    
    return ht

def concatToNumpy(A):
    blocks = A.ndarray.collect()
    big_mat = np.concatenate(blocks, axis=0)
    block_shape = blocks[0].shape
    num_blocks = len(blocks)
    assert big_mat.shape == (num_blocks*block_shape[0], block_shape[1])
    return big_mat

def ndarray_to_table(chunked_arr):
    structs = [hl.struct(row_group_number = idx, ndarray = block)
               for idx, block in enumerate(chunked_arr)]
    ht = hl.Table.parallelize(structs)
    ht = ht.key_by('row_group_number')
    return ht

# function to multiply two blocks, given the two blocks
# returns struct in form of array but not ndarray, includes the shape in the struct
# to change the result product directly back into a ndarray we need to use from_column_major
def block_product(left, right):
    product = left @ right
    n_rows, n_cols = product.shape
    return hl.struct(
        shape=product.shape,
        block=hl.range(hl.int(n_rows * n_cols)).map(
            lambda absolute: product[absolute % n_rows, absolute // n_rows]))

# takes in output of block_product
def block_aggregate(prod):
    shape = prod.shape
    block = prod.block
    return hl.nd.from_column_major(
        hl.agg.array_sum(block),
        hl.agg.take(shape, 1)[0])

# returns flat array
def to_column_major(ndarray):
    n_rows, n_cols = ndarray.shape
    return hl.range(hl.int(n_rows * n_cols)).map(
        lambda absolute: ndarray[absolute % n_rows, absolute // n_rows])

# hl.nd.from_column_major(thing.the_sum, thing.the_shape)

## Blanczos Algorithm

In [343]:
# Algorithm step: multiplying H0 = A @ G

# METHOD
# Multiply a row-blocked matrix by a local non-blocked matrix
# First step of algorithm

# Usage:
# assumes blocks in blocked matrix are named ndarray
# A is a table, B is a Hail ndarray
# returns a table

# Example:
# H0 = matmul_rowblocked_nonblocked(data, G)

def matmul_rowblocked_nonblocked(A, B):
    temp = A.annotate_globals(mat = B)
    temp = temp.annotate(ndarray = temp.ndarray @ temp.mat)
    temp = temp.select(temp.ndarray)
    temp = temp.drop(temp.mat)
    return temp


# Algorithm step: intermediate operation of multiplying At @ (A @ G) = At @ H0

# METHOD
# Multiply a column-blocked matrix by a row-blocked matrix 
# as a blockmatrix multiplcation and then sum
# Second step of algorithm

# Usage:
# pass in matrix A normally, blocked in rows - this specifically expects A to need to be transposed
# assumes blocks in blocked matrix are named ndarray
# A and B are both tables
# returns a hail ndarray

# Example:
# G1 = matmul_colblocked_rowblocked(data, H0)

def matmul_colblocked_rowblocked(A, B):
    temp = A.transmute(ndarray = block_product(A.ndarray.transpose(), B[A.row_group_number].ndarray))
    result_arr_sum = temp.aggregate(block_aggregate(temp.ndarray))
#     blocksA = A.collect()
#     blockAshape = blocksA[0].ndarray.shape
#     print('A block shape (gets transposed)', blockAshape)
#     blockBshape = B.take(1)[0].ndarray.shape
#     print('H block shape', blockBshape)
#     print(blockAshape[1], blockBshape[1])
#     print('shape of blocks in temp', temp.take(1)[0].ndarray.shape)
    return result_arr_sum


def matmul_nonblocked_rowblocked(B, A):
    pass


In [339]:
# Algorithm step: perform QR decomposition of Hq and compute T = Q^T @ A
# Perform QR decomposition of a row-blocked matrix
# Third and fourth step of algorithm

def computeNextH(A, H):
    nextG = matmul_colblocked_rowblocked(A, H)
    return matmul_rowblocked_nonblocked(A, nextG)


# RETIRED FUNCTIONS FOR NOW


# def processH(H, A):
    
#     # perform QR decomposition on unblocked version of H
#     arr_H = concatToNumpy(H)
#     Q, R = np.linalg.qr(arr_H)
#     # assert(Q.shape == (m, (q+1)*l))
    
#     # block Q's rows into the same number of blocks that A has
#     num_blocks = A.count()
#     group_size_Q = Q.shape[0] // num_blocks
#     #assert group_size_Q * num_blocks == m
#     blocked_Q_table = ndarray_to_table(chunk_ndarray(Q, group_size_Q))
    
#     T = matmul_colblocked_rowblocked(blocked_Q_table, A)
#     U, S, W = np.linalg.svd(T, full_matrices=False)
#     V = matmul_rowblocked_nonblocked(blocked_Q_table, U)
    
#     return V, S, W


# Algorithm step: compute SVD of T such that T = USW^T
# 5th step of algorithm

# def processT(T):
#     U, S, W = np.linalg.svd(T, full_matrices=False)
#     # assert(U.shape == (l, (q+1)*l))
#     # assert(S.shape == ((q+1)*l,))
#     # assert(W.shape == ((q+1)*l, n))
#     return U, S, W


# Algorithm step: multiply V = Q @ W
# 6th step of algorithm and last step

# def computeV(U, S, W, Q):
#     V = matmul_rowblocked_nonblocked(Q, U)
#     return V, S, W

In [458]:
def hailBlanczos(A, G, m, n, k, l, q):
    
    assert l > k
    #assert (q+1)*l <= (n - k)
    assert n <= m
    
    Hi = matmul_rowblocked_nonblocked(A, G)
    npH = concatToNumpy(Hi)
    for j in range(0, q):
        Hj = computeNextH(A, Hi)
        npH = np.concatenate((npH, concatToNumpy(Hj)), axis=1)
        Hi = Hj
    
    assert npH.shape == (m, (q+1)*l)
    # perform QR decomposition on unblocked version of H
    Q, R = np.linalg.qr(npH)
    assert Q.shape == (m, (q+1)*l)
    
    # block Q's rows into the same number of blocks that A has
    num_blocks = A.count()
    group_size_Q = Q.shape[0] // num_blocks
    #assert group_size_Q * num_blocks == m
    blocked_Q_table = ndarray_to_table(chunk_ndarray(Q, group_size_Q))
    
    T = matmul_colblocked_rowblocked(blocked_Q_table, A)
    assert(T.shape == ((q+1)*l), n)
    U, S, W = np.linalg.svd(T, full_matrices=False)
    print(U.shape)
    print(S.shape)
    print(W.shape)
#     assert U.shape == ((q+1)*l, n)
#     assert S.shape == (n,)
#     assert W.shape == (n, n)
    
    sing_val = S[k]
    
    V = matmul_rowblocked_nonblocked(blocked_Q_table, U)
    arr_V = concatToNumpy(V)
    
    truncV = arr_V[:,:k]
    truncS = S[:k]
    truncW = W[:k,:]
    
    bound = blanczosErrorB(truncV, np.diag(truncS), truncW.transpose(), m, n, k, q, concatToNumpy(A), sing_val)
    print("Satisfies Blanczos error bound equation 4.3 if C=1: ", bound)
    
    print(npH)
    print(npH.shape)
    
    return truncV, truncS, truncW, sing_val, Q

# def hailBlanczos(A, G):
#     H0 = matmul_rowblocked_nonblocked(A, G)
#     T, Q, table_Q = processH(H0, A)
#     u, s, w = processT(T)
#     V, S, W = computeV(u, s, w, table_Q)
#     return V, S, W

In [348]:
# T, Q, table_Q = processH(H0, data)
# print('Q shape', Q.shape)

# u, s, w = processT(T)
# V, S, W = computeV(u, s, w, table_Q)
# arr_V = concatToNumpy(V)
# print('V shape:', arr_V.shape, 'S shape:', S.shape, 'W shape:', W.shape)

## NumPy implementation from other notebook

In [457]:
# ALL TRANSPOSED FROM ABOVE IMPLEMENTATION

def hwe_normalize(call_expr):
    mt = call_expr._indices.source
    mt = mt.select_entries(__gt=call_expr.n_alt_alleles())
    mt = mt.annotate_rows(__AC=hl.agg.sum(mt.__gt),
                          __n_called=hl.agg.count_where(hl.is_defined(mt.__gt)))
    mt = mt.filter_rows((mt.__AC > 0) & (mt.__AC < 2 * mt.__n_called))

    n_variants = mt.count_rows()
    if n_variants == 0:
        raise FatalError("hwe_normalized: found 0 variants after filtering out monomorphic sites.")

    mt = mt.annotate_rows(__mean_gt=mt.__AC / mt.__n_called)
    mt = mt.annotate_rows(
        __hwe_scaled_std_dev=hl.sqrt(mt.__mean_gt * (2 - mt.__mean_gt) * n_variants / 2))
    mt = mt.unfilter_entries()

    normalized_gt = hl.or_else((mt.__gt - mt.__mean_gt) / mt.__hwe_scaled_std_dev, 0.0)
    return normalized_gt

#Blanczos paper error bound 4.23
def blanczosErrorA(U, S, V, m, n, k, q, A, k1th_sing_val):
    norm_diff = np.linalg.norm(A - U @ S @ V.transpose())
    bound = 100 * l * (((m-k)/l) ** (1/(4*q + 2))) * k1th_sing_val
    print('value:', norm_diff, 'bound/upper limit:', bound)
    return norm_diff <= bound

def blanczosErrorB(U, S, V, m, n, k, q, A, k1th_sing_val):
    C = 1
    norm_diff = np.linalg.norm(A - U @ S @ V.transpose())
    bound = C * (m ** (1/(4*q))) * k1th_sing_val
    satisfyingC = norm_diff / bound
    print('difference A - USV:', norm_diff, 'bound/upper limit:', bound)
    print('C constant needed to satisfy bound:', satisfyingC)
    return norm_diff <= bound

def normQQtA(A, Q):
    return np.linalg.norm(A - Q @ Q.transpose() @ A)

def numpyBlanczos(A, m, n, k, l, q, G):

    assert l > k
    assert (q+1)*l <= (m - k)
    assert m <= n

    # G = np.random.normal(0, 1, (l, m))
    R = G @ A
    # AtA = A.transpose() @ A
    listR = [R]
    for i in range(0, q):
        Ri = (listR[i] @ A.transpose()) @ A
        listR.append(Ri)
        R = np.concatenate((R, Ri), axis=0)

    assert(R.shape == ((q+1)*l), n)
    (Q, S) = np.linalg.qr(R.transpose())
    assert(Q.shape == (n, (q+1)*l))
    assert(S.shape == ((q+1)*l, (q+1)*l))

    T = A @ Q
    assert(T.shape == (m, (q+1)*l))
    
    (Tu, Ts, Tw) = np.linalg.svd(T, full_matrices=False)
    assert Tu.shape, (m, (q+1)*l)
    print(Ts.shape)
    assert Ts.shape == ((q+1)*l,)
    assert Tw.shape == ((q+1)*l, (q+1)*l)
    
    sing_val = Ts[k]
    
    V = Q @ Tw
    
    bound = blanczosErrorB(Tu, np.diag(Ts), V, m, n, k, q, A, sing_val)
    print("Satisfies Blanczos error bound equation 4.3 if C=1: ", bound)
    
    return Tu[:,:k], Ts[:k], V[:k,:], sing_val, Q


# mt = hl.balding_nichols_model(3, 10000, 1000)
# norm_gt = hwe_normalize(mt.GT)
# np_matrix = hl.linalg.BlockMatrix.from_entry_expr(norm_gt).to_numpy()

# npA = np.asmatrix(np_matrix)
# print(npA.shape)
# npm, npn = npA.shape
# npk = 50
# npl = npk + 2
# npq = 0

# (blanczosU, blanczosS, blanczosV) = blanczosSVD(npA, npm, npn, npk, npl, npq)

## Run NumPy and Hail implementations on the same data

In [446]:
def makeSharedData(model_input, block_size):
    
    # we should have m > n for hail implementation
    mt = hl.balding_nichols_model(*model_input)
    
    mt.write("balding_nichols_data.mt")
    mt = hl.read_matrix_table("balding_nichols_data.mt")
    
    mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 
    table = mt.localize_entries("ent", "sample")
    table = matrix_table_to_table_of_ndarrays(mt.n_alt, block_size, tmp_path='/tmp/test_table.ht')
    # table = table.key_by(hl.int32(table.row_group_number))

    # for numpy implementation we want transposed version so m < n
    np_matrix = np.asmatrix(concatToNumpy(table).transpose())

    return table, np_matrix

m = 1500
n = 500
block_size = 4
hailA, numpyA = makeSharedData((3, n, m), block_size)

# doesn't account for differences due to transposing
# hailV not necessarily supposed to be the same as blanczosU

2020-07-30 16:18:23 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 500 samples, and 1500 variants...
2020-07-30 16:18:24 Hail: INFO: Coerced sorted dataset
2020-07-30 16:18:25 Hail: INFO: wrote matrix table with 1500 rows and 500 columns in 8 partitions to balding_nichols_data.mt
2020-07-30 16:18:26 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-07-30 16:18:28 Hail: INFO: wrote table with 375 rows in 8 partitions to /tmp/test_table.ht


In [453]:
k = 30
l = k + 2
q = 5
G = np.random.normal(0, 1, (n,l))
(q+1)*l

192

In [460]:
hailV, hailS, hailW, hailSingVal, hailQ = hailBlanczos(hailA, hl.nd.array(G), m, n, k, l, q)

2020-07-30 16:53:35 Hail: INFO: Coerced sorted dataset


(192, 192)
(192,)
(192, 500)


2020-07-30 16:53:42 Hail: INFO: Coerced sorted dataset


difference A - USV: 479.95933512322597 bound/upper limit: 47.620395220267454
C constant needed to satisfy bound: 10.078860809600192
Satisfies Blanczos error bound equation 4.3 if C=1:  False
[[ 1.85655945e+01  4.33744259e+01 -7.73530726e+00 ...  5.33111350e+30
  -1.92248338e+31  7.08423999e+30]
 [ 1.83043495e+01  2.61983241e+01  2.74185092e+01 ...  6.99198112e+30
  -2.52141836e+31  9.29128075e+30]
 [ 1.85651598e+01  1.90451970e+01  2.78771242e+01 ...  6.12566794e+30
  -2.20901221e+31  8.14008214e+30]
 ...
 [ 7.51548651e+00 -6.03492638e+00 -1.84074682e+00 ...  6.62240769e+29
  -2.38814437e+30  8.80017379e+29]
 [ 1.95007764e+01  1.92256835e+01  3.54587306e+01 ...  8.63117409e+30
  -3.11253713e+31  1.14695192e+31]
 [ 2.97016606e+01  2.24638550e+01  1.65317403e+01 ...  6.93128718e+30
  -2.49953117e+31  9.21062773e+30]]
(1500, 192)


In [461]:
numpyU, numpyS, numpyV, numpySingVal, numpyQ = numpyBlanczos(numpyA, n, m, k, l, q, G.transpose())

(192,)
difference A - USV: 1344.2996326997277 bound/upper limit: 45.04738421678012
C constant needed to satisfy bound: 29.841902167517578
Satisfies Blanczos error bound equation 4.3 if C=1:  False


In [455]:
print('largest singular value from hail:', hailS[0])
print('smallest singular value from hail:', hailS[k-1])
print(hailS.shape)
print(numpyS.shape)
hailS - numpyS

largest singular value from hail: 967.4853701356503
smallest singular value from hail: 33.11313403874567
(30,)
(30,)


array([ 0.00000000e+00, -1.42108547e-13, -8.52651283e-14,  4.18694091e-03,
        1.41952204e-02,  1.36519952e-02,  4.86290621e-02, -9.52093757e-03,
       -4.78784745e-02,  2.25612226e-02, -4.82960177e-02, -2.11879055e-02,
       -5.18508218e-02, -9.17235545e-02, -5.27615972e-02,  1.26308488e-01,
        4.21951592e-02, -9.54389216e-03, -1.08156697e-01, -5.09682138e-02,
       -7.54866140e-02, -4.04461179e-02, -7.88173382e-02,  5.55095613e-02,
        1.57699216e-02,  2.79714658e-02,  8.01911034e-03,  1.13023507e-01,
       -1.06120540e-02, -1.34380798e-02])

In [465]:
normQQtA(numpyA.transpose(), hailQ)

392.6023860761228

In [467]:
print(numpyA.shape)
print(numpyQ.shape)
normQQtA(numpyA, numpyQ.transpose())

(500, 1500)
(1500, 192)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 500 is different from 192)